In [1]:
# Cela 1
# PASO 1: PREPARACIÓN DE DATOS
#

import sqlite3
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def load_data_from_db(db_name="fightaging_articles.db"):
    """Carga los artículos de la base de datos a un DataFrame de Pandas."""
    try:
        conn = sqlite3.connect(db_name)
        df = pd.read_sql_query("SELECT * FROM articles", conn)
        conn.close()
        df['publish_date'] = pd.to_datetime(df['publish_date'])
        print(f"Se cargaron {len(df)} artículos de la base de datos.")
        return df
    except Exception as e:
        print(f"Error al cargar datos: {e}")
        return pd.DataFrame()

def clean_text(text):
    """
    Función para limpiar el texto:
    1. Convierte a minúsculas.
    2. Elimina puntuación y números.
    3. Separa el texto en palabras (tokenización).
    4. Elimina "stop words" (palabras comunes en inglés).
    5. Elimina palabras muy cortas (1 o 2 letras).
    """
    # 1. Minúsculas
    text = text.lower()
    
    # 2. Eliminar puntuación y números usando expresiones regulares
    text = re.sub(r'[^a-z\s]', '', text)
    
    # 3. Tokenización (dividir en palabras)
    tokens = word_tokenize(text)
    
    # 4. Eliminar stop words
    stop_words = set(stopwords.words('english'))
    
    # Añadimos algunas palabras personalizadas que son muy comunes en este contexto
    # pero que no aportan mucho significado para diferenciar temas.
    custom_stop_words = [
        'welcome', 'fight', 'also', 'study', 'research', 'aging', 'age', 'may', 
        'however', 'results', 'data', 'found', 'open', 'access', 
        'great', 'deal', 'one', 'even', 'work', 'life', 'people', 'years', 'healthy',
        'life', 'longevity', 'human'
    ]
    
    stop_words.update(custom_stop_words)
    
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # 5. Eliminar palabras muy cortas
    filtered_tokens = [word for word in filtered_tokens if len(word) > 2]
    
    # Unir las palabras de nuevo en un solo string
    return " ".join(filtered_tokens)


# --- Ejecución del Paso 1 ---

if __name__ == "__main__":
    articles_df = load_data_from_db()

    if not articles_df.empty:
        # Combinamos título y cuerpo para un análisis más completo
        articles_df['full_text'] = articles_df['title'] + ' ' + articles_df['body']
        
        print("\nLimpiando y preprocesando el texto de los artículos...")
        # Aplicamos la función de limpieza a cada artículo
        articles_df['cleaned_text'] = articles_df['full_text'].apply(clean_text)
        
        # Mostramos una comparación del antes y el después
        print("\nEjemplo de limpieza:")
        print("--- TEXTO ORIGINAL ---")
        print(articles_df['full_text'].iloc[1][:500]) # Muestra los primeros 500 caracteres
        print("\n--- TEXTO LIMPIO ---")
        print(articles_df['cleaned_text'].iloc[1][:500])
        
        # Guardamos el DataFrame procesado para no tener que repetir este paso
        articles_df.to_pickle("articles_cleaned.pkl")
        print("\nDataFrame con texto limpio guardado en 'articles_cleaned.pkl'")

Se cargaron 18753 artículos de la base de datos.

Limpiando y preprocesando el texto de los artículos...

Ejemplo de limpieza:
--- TEXTO ORIGINAL ---
The Fight Aging! Disclaimer Please read this disclaimer carefully. It is a commonsense statement that should apply to all health information you find online. Your health is valuable and easily damaged. What is good advice for one person may not be good advice for another: people vary considerably in health matters. Information provided on Fight Aging! should always be discussed with a qualified physician. It is not intended to replace the relationship between you and your physician. It is recomm

--- TEXTO LIMPIO ---
disclaimer please read disclaimer carefully commonsense statement apply health information find online health valuable easily damaged good advice person good advice another vary considerably health matters information provided always discussed qualified physician intended replace relationship physician recommended follow topi

In [2]:
# Celda 2
# PASO 2: VECTORIZACIÓN
#

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Cargamos el DataFrame que limpiamos en el paso anterior
try:
    articles_df = pd.read_pickle("articles_cleaned.pkl")
    print("DataFrame con texto limpio cargado correctamente.")
except FileNotFoundError:
    print("Error: El archivo 'articles_cleaned.pkl' no fue encontrado.")
    print("Por favor, ejecuta primero el script del Paso 1 para generar este archivo.")
    exit()

# Nos aseguramos de que no haya textos vacíos que puedan causar problemas
articles_df.dropna(subset=['cleaned_text'], inplace=True)
corpus = articles_df['cleaned_text']

# --- Vectorización con TF-IDF ---
# TF-IDF es una técnica para convertir la importancia de una palabra en un documento en un número
# max_df=0.95: ignora palabras que aparecen en más del 95% de los documentos (demasiado comunes)
# min_df=5: ignora palabras que aparecen en menos de 5 documentos (demasiado raras)
# stop_words='english': una capa extra de limpieza
vectorizer = TfidfVectorizer(max_df=0.95, min_df=5, stop_words='english')

print("\nConvirtiendo el texto a una matriz numérica con TF-IDF...")
tfidf_matrix = vectorizer.fit_transform(corpus)

# La matriz tfidf_matrix contiene la representación numérica de nuestros artículos.
# 'feature_names' son las palabras (el vocabulario) que el vectorizador ha aprendido.
feature_names = vectorizer.get_feature_names_out()

print(f"La matriz TF-IDF ha sido creada.")
print(f"Forma de la matriz: {tfidf_matrix.shape[0]} artículos y {tfidf_matrix.shape[1]} palabras únicas.")
print("\nEjemplo de las primeras 10 palabras del vocabulario aprendido:")
print(feature_names[:10])


DataFrame con texto limpio cargado correctamente.

Convirtiendo el texto a una matriz numérica con TF-IDF...
La matriz TF-IDF ha sido creada.
Forma de la matriz: 18753 artículos y 22919 palabras únicas.

Ejemplo de las primeras 10 palabras del vocabulario aprendido:
['aaas' 'aak' 'aaron' 'aarp' 'aav' 'aavmediated' 'aavs' 'abandon'
 'abandoned' 'abandoning']


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import joblib  # Librería para guardar y cargar modelos
import os      # Para verificar si el archivo existe

# --- Función para mostrar los tópicos (sin cambios) ---
def display_topics(model, feature_names, n_top_words):
    print("\n--- TÓPICOS DESCUBIERTOS POR EL MODELO LDA ---")
    for topic_idx, topic in enumerate(model.components_):
        message = f"Tópico #{topic_idx}: "
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print("------------------------------------------")

# --- Aplicamos el Modelo LDA (Paso 3 con lógica de guardado) ---

# Definimos el nombre del archivo donde se guardará el modelo
lda_model_file = 'lda_model.joblib'

# Definimos los parámetros del modelo
n_topics = 20
n_top_words = 20

# --- LÓGICA PARA CARGAR O ENTRENAR EL MODELO ---
if os.path.exists(lda_model_file):
    print(f"✅ Modelo LDA entrenado encontrado. Cargando desde '{lda_model_file}'...")
    lda = joblib.load(lda_model_file)
else:
    # Si el modelo no existe, lo entrenamos
    print(f"Modelo no encontrado. Entrenando un nuevo modelo LDA para encontrar {n_topics} tópicos...")
    
    # NOTA: Este bloque asume que 'tfidf_matrix' del Paso 2 está en memoria.
    
    # Creamos y entrenamos el modelo LDA
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(tfidf_matrix)
    
    # Guardamos el modelo recién entrenado para la próxima vez
    print(f"✅ Modelo entrenado. Guardando en '{lda_model_file}'...")
    joblib.dump(lda, lda_model_file)

# --- Mostramos los tópicos encontrados ---
# Esta parte se ejecuta siempre, ya sea con el modelo cargado o el recién entrenado.
# NOTA: Asume que 'feature_names' del Paso 2 está en memoria.
display_topics(lda, feature_names, n_top_words)

✅ Modelo LDA entrenado encontrado. Cargando desde 'lda_model.joblib'...

--- TÓPICOS DESCUBIERTOS POR EL MODELO LDA ---
Tópico #0: theraputic cloning ban cisd manhattan httpwwwbiomedcentralcomnews camr mcat hwang criminalize treaty cordis criminalizing httpdbscordislucgibinsrchidadbcallernhpennewsactiondsessionrcnenrcnid httpwwwcamradvocacyorgfastactionnewsaspid suk brownback deuterium manf crap
Tópico #1: brain alzheimers disease amyloid neurons gut blood cells microglia inflammation cognitive tau protein memory mice immune microbiome neurodegenerative inflammatory researchers
Tópico #2: science extension medical medicine sens foundation funding future new time like stem progress good cryonics think world way live article
Tópico #3: printing organs vessels artificial tissue engineering scaffold xenotransplantation blood bioprinting pig organ printed oocytes decellularization scaffolds pshc printer trachea nanofibers
Tópico #4: hair progeria lamin progerin disc hgps reeve follicle tran